In [4]:
### parameters
place = 'tel_aviv'
feature = 'slope'

In [6]:
from rasterio.sample import sample_gen
import rasterio



import geopandas as gpd
from shapely.geometry import LineString, Point
from tqdm import tqdm  # for progress bar





import pickle
import glob
import numpy as np
import os
from pathlib import Path
from scipy.ndimage import generic_filter
import warnings
warnings.filterwarnings(action='ignore')
crs_prj = 'EPSG:2039'


# Get the current working directory (e.g., the folder you're running from)
cwd = Path().resolve()

# Get the parent directory
parent_folder = f'{cwd.parent}/places/{place}'
data_folder = f'{parent_folder}/shp'
os.makedirs(f'{parent_folder}',exist_ok=True)
os.makedirs(f'{parent_folder}/shp',exist_ok=True)
os.makedirs(f'{parent_folder}/shp/{feature}',exist_ok=True)
detail_folder = f'{data_folder}/{feature}'

In [19]:



# === Load Data ===
streets_gdf = gpd.read_file(f'{data_folder}/streets.shp')[['oidrechov', 'geometry']]  # Must include 'oidrechov'
dem = rasterio.open(f'{detail_folder}/itm_raster.tif')

# === Function to split line into ~10m segments ===
def split_line_to_segments(line, segment_length=10):
    num_segments = int(np.ceil(line.length / segment_length))
    if num_segments < 1:
        return [line]
    points = [line.interpolate(float(i) / num_segments, normalized=True) for i in range(num_segments + 1)]
    return [LineString([points[i], points[i+1]]) for i in range(len(points) - 1)]

# === Function to get elevation from DEM ===
def get_elevation(pt, dem):
    coords = [(pt.x, pt.y)]
    elevation = list(rasterio.sample.sample_gen(dem, coords))[0][0]
    return elevation

# === Process all streets ===
results = []
my_break=False
for idx, row in  tqdm(streets_gdf.iterrows(), total=len(streets_gdf), desc="Processing streets"):
    oid = row['oidrechov']
    geom = row.geometry

    if geom.length < 0.1:
        continue  # Skip tiny geometries

    segments = split_line_to_segments(geom, segment_length=10)

    slopes = []
    for segment in segments:
        start_pt = Point(segment.coords[0])
        end_pt = Point(segment.coords[1])
        try:
            z1 = get_elevation(start_pt, dem)
            z2 = get_elevation(end_pt, dem)
        except:
            print(f'there is an error in {segment}')
            my_break=True
            break

        length = segment.length
        if length > 0:
            slope = abs(z2 - z1) / length
            slopes.append(slope)
    if my_break:
        break
    # Average slope for the street
    if slopes:
        avg_slope = np.mean(slopes)
        results.append({'oidrechov': oid, 'avg_slope': avg_slope})
# === Create result GeoDataFrame ===
slope_df = gpd.GeoDataFrame(results)
streets_gdf.merge(slope_df, on="oidrechov", how="left").to_file(f'{detail_folder}/{feature}.shp')

Processing streets: 100%|██████████| 8751/8751 [00:57<00:00, 151.31it/s]
